In [1]:
import pandas as pd

In [2]:
def pct(x, y):
    return (x - y) / ((x + y) / 2) * 100

In [3]:
def oracle(df, opt):
    print('Energy overhead of the dynamic approach compared to an oracle-based approach')

    mt = df[df['threads'] == 'mt'].copy()
    df = df[(df['threads'] != 'mt') & (df['threads'] != 'rt')].copy()

    mt['runtime'] = mt.apply(lambda row: pct(row['runtime'], df[(df['size'] == row['size']) & (df['threads'] == opt[row['size']])].iloc[0]['runtime']), axis=1)
    mt['energy']  = mt.apply(lambda row: pct(row['energy'],  df[(df['size'] == row['size']) & (df['threads'] == opt[row['size']])].iloc[0]['energy']), axis=1)

    return mt.groupby('size').mean(numeric_only=True).round(0).astype(int)

In [4]:
def runtime(df):
    print('\nEnergy overhead of the dynamic approach compared to a runtime-based approach')

    mt = df[df['threads'] == 'mt'].copy()
    rt = df[df['threads'] == 'rt'].copy()

    mt['runtime'] = mt.apply(lambda row: pct(row['runtime'], rt[rt['size'] == row['size']].iloc[0]['runtime']), axis=1)
    mt['energy']  = mt.apply(lambda row: pct(row['energy'],  rt[rt['size'] == row['size']].iloc[0]['energy']), axis=1)

    return mt.groupby('size').mean(numeric_only=True).round(0).astype(int)

In [5]:
def mean(df):
    print('\nAverage energy overhead of the dynamic approach compared to a static approach')
    
    mt = df[df['threads'] == 'mt'].copy()
    df = df[(df['threads'] != 'mt') & (df['threads'] != 'rt')].copy()

    # Normalise runtime and energy consumption
    df['runtime'] = df.apply(lambda row: pct(mt[mt['size'] == row['size']].iloc[0]['runtime'], row['runtime']), axis=1)
    df['energy']  = df.apply(lambda row: pct(mt[mt['size'] == row['size']].iloc[0]['energy'],  row['energy']), axis=1)

    # Drop unused columns and rows
    df = df.drop('size', axis=1)
    df = df[df['threads'] != 'mt']
    df['threads'] = df['threads'].astype(int)

    # Mean per thread-count
    return df.groupby('threads').mean(numeric_only=True).round(0).astype(int)

---

In [6]:
df = pd.read_csv('data/compare_nbody.csv')
df = df.drop(['runtimesd', 'energysd'], axis=1)
print(oracle(df, { 10000: '16', 25000: '14', 40000: '12' }))
print(runtime(df))
print(mean(df))

Energy overhead of the dynamic approach compared to an oracle-based approach
       runtime  energy
size                  
10000        7       4
25000       -3       3
40000        2       4

Energy overhead of the dynamic approach compared to a runtime-based approach
       runtime  energy
size                  
10000       -6       0
25000       -6       0
40000       10       1

Average energy overhead of the dynamic approach compared to a static approach
         runtime  energy
threads                 
1           -148     -80
8            -51     -11
12           -13       0
14             1       2
16             6      -1


In [7]:
df = pd.read_csv('data/compare_stencil.csv')
df = df.drop(['runtimesd', 'energysd'], axis=1)
print(oracle(df, { 10000: '16', 25000: '14', 40000: '12' }))
print(runtime(df))
print(mean(df))

Energy overhead of the dynamic approach compared to an oracle-based approach
       runtime  energy
size                  
10000        4       5
25000        5       5
40000       33      15

Energy overhead of the dynamic approach compared to a runtime-based approach
       runtime  energy
size                  
10000       -8      -2
25000       -3      -4
40000       25       7

Average energy overhead of the dynamic approach compared to a static approach
         runtime  energy
threads                 
1           -137     -79
8            -39     -10
12             0       4
14             8       3
16             7      -5


In [8]:
df = pd.read_csv('data/compare_matmul.csv')
df = df.drop(['runtimesd', 'energysd'], axis=1)
print(oracle(df, { 500: '8', 1000: '8', 1500: '8' }))
print(runtime(df))
print(mean(df))

Energy overhead of the dynamic approach compared to an oracle-based approach
      runtime  energy
size                 
500         8       8
1000       93      46
1500      100      68

Energy overhead of the dynamic approach compared to a runtime-based approach
      runtime  energy
size                 
500       -13      -7
1000       41      10
1500       14      -3

Average energy overhead of the dynamic approach compared to a static approach
         runtime  energy
threads                 
1           -110     -55
8             67      41
12           -40     -58
16           -29     -49


In [9]:
df = pd.read_csv('data/compare_rust.csv')
df = df.drop(['runtimesd', 'energysd'], axis=1)
print(oracle(df, { 500: '16', 1000: '16', 1500: '8' }))
print(runtime(df))
print(mean(df))

Energy overhead of the dynamic approach compared to an oracle-based approach
      runtime  energy
size                 
500         1       1
1000        0       1
1500       16      10

Energy overhead of the dynamic approach compared to a runtime-based approach
      runtime  energy
size                 
500        -3      -2
1000       -1      -1
1500       -2       1

Average energy overhead of the dynamic approach compared to a static approach
         runtime  energy
threads                 
1           -157    -100
8             -3       1
12            -4      -2
16            -7      -7
